# REQUIRED PACKAGE INSTALL

In [10]:
# pip install function
import subprocess
import sys

def install(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    except:
        print("No such package")

In [11]:
# pip install packages
install('pytesseract')
install('pdf2image')
install('PyPDF2')
install('PyMuPDF')

In [1]:
import pytesseract
from pdf2image import convert_from_path
import PyPDF2
import io
import fitz # imports the pymupdf library

In [13]:
# uncomment the code below if you don't have tesseract added to path
# pytesseract.pytesseract.tesseract_cmd = r"Tesseract-OCR/tesseract.exe"

# INPUT LAYER

In [19]:
# pdf_doc = r'files/woodBoy.pdf'
pdf_doc = r'files/Brabender_P1.pdf'

# PARSING LAYER

In [5]:
# imagePDF_to_text function
def imagePDF_to_text(file):
    # poppler path
    poppler_path=r"Release-24.02.0-0/poppler-24.02.0/Library/bin"
    # convert to image
    images = convert_from_path(file, poppler_path=poppler_path)
    
    pdf_writer = PyPDF2.PdfWriter()
    page_text = ''
    for image in images:
        page = pytesseract.image_to_pdf_or_hocr(image, extension='pdf')
        pdf = PyPDF2.PdfReader(io.BytesIO(page))

        page_text += (pdf.pages[0]).extract_text()
        pdf_writer.add_page(pdf.pages[0])
    
    return page_text

In [6]:
# PDF_to_text function
def PDF_to_text(file):
  try:
    # works only for text-based pdf
    doc = fitz.open(file) # open a document
    for page in doc: # iterate the document pages
      text = page.get_text() # get plain text encoded as UTF-8
      if not text.strip():
        raise Exception('PDF is a scanned document')
    print('Done!')
    return text
  except Exception as e:
    print(f"Error while processing PDF: {e}")
    print('Running OCR engine...')
    text = imagePDF_to_text(file)
    print('Done!\n')
    return text

In [20]:
text = PDF_to_text(pdf_doc)
# print(text)

Error while processing PDF: PDF is a scanned document
Running OCR engine...
Done!



In [18]:
# export the searchable PDF to searchable.pdf
# out_file = f'{pdf_doc[:-4]}_searchable.pdf'
# with open(out_file, "wb") as f:
#     pdf_writer.write(f)

In [28]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
# Use a facebook pipeline as a high-level helper
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [22]:
len(text)

117935

In [23]:
# from functools import reduce
# text = text.replace("\n", " ")
# words = text.split(" ")
# first_256_w = words[:256]
# new_sentences = reduce(lambda x, y: x + " " + y, first_256_w)

In [25]:
len(first_256_w)
new_sentences = new_sentences.replace("\n", " ")

256

In [15]:
# # Initialize variables
# max_length = 512  # Maximum sequence length supported by the model
# segments = []
# current_segment = []

# # Split the input into segments
# for word in words:
#     current_segment.append(word)
#     if len(current_segment) == max_length:
#         segments.append(current_segment)
#         current_segment = []
# # Add the last segment if it's not empty
# if current_segment:
#     segments.append(current_segment)


# # Process each segment separately
# for segment in segments:
#     # Convert segment tokens back to text
#     # segment_text = tokenizer.convert_tokens_to_string(segment)
#     # Encode segment
#     # inputs = tokenizer(segment_text, return_tensors='pt')
#     # Forward pass through the model
#     print(text+'\n')
#     outputs = summarizer(text, max_length=100, min_length=50, do_sample=False)
#     # outputs = model(**inputs)
#     print(outputs)
#     # Process model outputs as needed


In [ ]:
# Split the input document into smaller segments
segments = text.split("\n\n")  # Split by paragraphs, adjust as needed

# Generate summaries for each segment
summaries = []
max_segment_length = 1024  # Max segment length for T5 model
for segment in segments:
    # Tokenize segment
    segment_tokens = tokenizer.encode(segment, return_tensors="pt", max_length=max_segment_length, truncation=True)
    # Generate summary for the segment
    summary_ids = model.generate(segment_tokens, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary)

# Combine summaries to form the final summary
final_summary = "\n".join(summaries)
print(final_summary)
